In [80]:
# setting up the enviroment

!pip install langchain_community
!pip install langchain_core
!pip install beautifulsoup4
!pip install langchain_text_splitters
!pip install langchain
!pip install langchainhub
!pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 14.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.9.0
    Uninstalling openai-1.9.0:
      Successfully uninstalled openai-1.9.0


In [93]:
!pip install -U langchain-community tavily-python

In [63]:
# setting up langsmith 

LANGCHAIN_TRACING_V2="true"
LANGCHAIN_API_KEY="..."

In [64]:
# setting up a local environment w/ Ollama and llama2

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [65]:
# We have this way to guide the answer by instructing the model 

from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [66]:
chain = prompt | llm 

In [67]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is a fantastic tool for testing purposes. Here are some ways in which Langsmith can assist with testing:\n\n1. Automated Testing: Langsmith provides a robust API for automating the creation and management of documentation. This means you can use Langsmith's API to automate the testing process, creating and executing tests faster and more efficiently than ever before.\n2. Consistency: One of the biggest challenges in testing is ensuring consistency across different test cases. With Langsmith, you can easily maintain consistency by using its built-in formatting and styling options to ensure that all test cases have a uniform structure and appearance.\n3. Collaboration: Langsmith allows multiple users to collaborate on documentation projects, making it easier to involve more people in the testing process. This means you can get input from a wider range of experts and stakeholders, lead

In [68]:
# parser to cast the input as string 

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [69]:
# redefine the cahin 

chain = prompt | llm | output_parser

In [70]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith is a versatile tool that can greatly assist in the testing process. Here are some ways Langsmith can help:\n\n1. Automated Testing: Langsmith's machine learning capabilities allow it to automatically generate test cases based on the documentation it has been trained on. This can save a tremendous amount of time and effort compared to manual testing, as it eliminates the need for human testers to write and execute test cases.\n2. Test Data Generation: Langsmith can also generate test data based on the documentation it has been trained on. This can be especially useful when testing complex systems or applications with numerous inputs and outputs. By generating test data automatically, Langsmith can help ensure that all possible scenarios are covered, reducing the likelihood of errors or bugs going undetected.\n3. Regression Testing: As a technical documentation writer, I understand th

In [71]:
# we will use a dataloader to make the model retrerive the data from the web (only some data will be used to answer the question)

# we need to install this for reasons that are not clear to me (apparently is a vector store)
!pip install faiss-cpu

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

In [72]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [73]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

# here we have embedded the documents and stored them in a vector store using documents = splitted text from the web data loder and embeddings = ollama embeddings

In [74]:
#Now that we have this data indexed in a vectorstore, we will create a retrieval chain. 

# THIS IS IMPORTANT BECAUSE, quote: 
'''
This chain will take an incoming question, look up relevant documents,
then pass those documents along with the original question into an LLM and ask it to answer the original question.
'''

# This means that the model will be able to answer the question by looking up the relevant documents in the vector store (INCLUDING CHAT HISTTORY)

from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [75]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...


Based on the provided context, LangSmith can help with testing in several ways:

1. Test Case Creation: LangSmith allows developers to create test cases and upload them in bulk, create them on the fly, or export them from application traces. This helps developers create and run tests on their LLM applications.
2. Evaluation Scoring: LangSmith provides native rendering of chat messages, functions, and retrieve documents, making it easier to score test results. Developers can also run custom evaluations (both LLM and heuristic based) to evaluate the performance of their applications.
3. Comparison View: LangSmith offers a user-friendly comparison view for test runs, allowing developers to track and diagnose regressions in test scores across multiple revisions of their application. This helps identify which variant is performing better.
4. Playground Environment: LangSmith provides a playground environment for rapid iteration and experimentation, allowing developers to quickly test out d

In [76]:
# now we will create a retrieval chain w/ chat history 

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [77]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most interesting runs and highlight edge cases that are causing problematic responses. LangSmith allows you to attach feedback scores to logged traces (oftentime

In [78]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [79]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Every playground run is logged in the system and can be used to create test cases or compare with other runs.Beta Testing\u200bBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it’s important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it’s breaking down in those cases. Both feedback collection and run annotation are critical for this workflow. This will help in curation of test cases that can help track regressions/improvements and development of automatic evaluations.Capturing Feedback\u200bWhen launching your application to an initial set of users, it’s important to gather human feedback on the responses it’s producing. This helps draw attention to the most i

### creating an agent (API, not local)




In [81]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

TAVILY_API_KEY=...

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

tools = [retriever_tool, search]


from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `openai_api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "how can langsmith help with testing?"})

agent_executor.invoke({"input": "what is the weather in SF?"})

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})


ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

## Testing itinerary for claire on retrival chain 

In [91]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x10e435710>)

In [90]:
from langchain_core.messages import HumanMessage, AIMessage
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt2 = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain2 = create_history_aware_retriever(llm, retriever, prompt2)

Claire = "Claire Dubois, Age**: 32 . Occupation**: Graphic Designer, Location**: Lyon, Interests**: Claire has a passion for contemporary art, indie music, and environmental activism. She frequents art galleries and music festivals and participates in local initiatives to promote sustainability, Personality**: Creative, open-minded, and socially conscious. She enjoys exploring new cultures and ideas and is always looking for ways to incorporate sustainability into her life and work."

chat_history2 = [HumanMessage(content=Claire), AIMessage(content="memorized!")]
retriever_chain2.invoke({
    "chat_history": chat_history2,
    "input": "reccomend me a trip for Claire"
})

TypeError: create_history_aware_retriever() missing 1 required positional argument: 'prompt'

### Setting up Tavily API for up to date data

In [ ]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [95]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

retriever.invoke("what year was breath of the wild released?")

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)
chain = (
    RunnablePassthrough.assign(context=(lambda x: x["question"]) | retriever)
    | prompt
    | ChatOpenAI(model="gpt-4-1106-preview")
    | StrOutputParser()
)